In [1]:
import networkx as nx;
import matplotlib.pyplot as plt
import pandas as pd;
import gurobipy as gp;
from gurobipy import GRB;
import csv;
import sys;
import time;
from datetime import datetime;
import math;
import random;
import itertools;

In [2]:
# Master


def Master(Z, bud, G, tau, s, t, it):
    
    M = A + 1;
    My = M;
    Md = 1e6;
    Mt = 1e6;
    
    model = gp.Model("Master-Problem");

    
    # Variables (Upper Level)
    eta = model.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    gamma = model.addVars(G.edges, vtype=GRB.BINARY);
    xBar = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    lamb = model.addVars(tau, vtype=GRB.BINARY); 
    
    x = {};
    y = {};
    alpha = {};
    theta = {};
    pi = {};
    delta = {};
    v = {};
    muTau = {};
    mu = {};
    
   
    
    # Constraints ("Upper Level")
    model.addConstr(gp.quicksum(gamma[i,j]*G.edges[i,j]['cost'] for i,j in G.edges) <= budget[b]);      
    
    model.addConstrs(gp.quicksum(xBar[j,i] for i in G.successors(j))
                 - gp.quicksum(xBar[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);       
    
    model.addConstr(xBar[t,s] == gp.quicksum(pow(2, tau[u])*lamb[u] for u in tau));
    
    for i,j in G.edges:
        model.addConstr(xBar[i,j] - (1-gamma[i,j])*G.edges[i,j]['capacity'] <= 0);  

    
    
    # Variables ("Lower Level")
    for n in range(len(Z)):
        x[n] = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
        y[n] = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
        alpha[n] = model.addVars(G.nodes, vtype=GRB.CONTINUOUS); 
        theta[n] = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
        pi[n] = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = -My, ub=0);
        delta[n] = model.addVar(vtype=GRB.CONTINUOUS, lb = -Md, ub=0);                               
        v[n] = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
        muTau[n] = model.addVars(tau, vtype=GRB.CONTINUOUS, ub = 0);
        mu[n] = model.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);                      
        
        
    # Constraints ("Lower Level")
        model.addConstrs(gp.quicksum(x[n][j,i] for i in G.successors(j)) 
                         - gp.quicksum(x[n][i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);           
    
        for i,j in G.edges:
            model.addConstr(x[n][i,j] - (1-gamma[i,j])*G.edges[i,j]['capacity'] <= 0);                      
            model.addConstr(v[n][i,j] <= theta[n][i,j]);                                                   
            model.addConstr(v[n][i,j] >= theta[n][i,j] - Mt*gamma[i,j]);
            model.addConstr(v[n][i,j] <= Mt*(1-gamma[i,j]));
            if G.edges[i,j]['special'] == 1:
                model.addConstr(x[n][i,j] + y[n][i,j] >= (1/M)*Z[n][i,j]);                                         
                model.addConstr(alpha[n][i] - alpha[n][j] + theta[n][i,j] + pi[n][i,j] >= 0);               
            elif i!=t:
                model.addConstr(alpha[n][i] - alpha[n][j] + theta[n][i,j] >= 0);                           
            
    
        model.addConstr(x[n][t,s] >= xBar[t,s]);                                                          
    
        model.addConstr(alpha[n][t] - alpha[n][s] + theta[n][t,s] + delta[n] >= 0);                                    
    
        model.addConstr(mu[n] == gp.quicksum(pow(2, tau[u])*muTau[n][u] for u in tau));
        
        for u in tau:
            model.addConstr(muTau[n][u] >= delta[n]);
            model.addConstr(muTau[n][u] >= -Md*lamb[u]);
            model.addConstr(muTau[n][u] <= delta[n] - Md*(lamb[u] - 1));
    
        model.addConstr(eta >= gp.quicksum(Z[n][i,j]*G.edges[i,j]['special'] for i,j in G.edges) +
                    gp.quicksum(G.edges[i,j]['capacity']*v[n][i,j] for i,j in G.edges) +
                    (1/M)*gp.quicksum(Z[n][i,j]*G.edges[i,j]['special']*pi[n][i,j] for i,j in G.edges) + mu[n]); 
    
        model.addConstr(-My*gp.quicksum(y[n][i,j]*G.edges[i,j]['special'] for i,j in G.edges) 
                    >= gp.quicksum(G.edges[i,j]['capacity']*v[n][i,j] for i,j in G.edges) + 
                    (1/M)*gp.quicksum(Z[n][i,j]*G.edges[i,j]['special']*pi[n][i,j] for i,j in G.edges) + mu[n]); 
    
     
    model.setObjective(eta, GRB.MINIMIZE);
    model.update();
    model.setParam("OutputFlag", 0);
    model.optimize();
    
    LB = model.objVal;
    
    print('Iteration: %g' %it);

    #print('LB (Master): %g' %LB);
                
    
    for i,j in G.edges:
        gammaStar[i,j] = gamma[i,j].x;

    
    #print('xBar_ts', 'flow :', xBar[t,s].x);
    #print('x_ts', 'flow :', x[n][t,s].x);
    
    return gammaStar, xBar[t,s].x, LB;    

In [3]:
# Sub-problem

def Sub(gammaStar, xBar_ts, G, t, s, UB):
    
   
    M = A+1;
    
    xStar = {};
    zStar = {};
    
    sub = gp.Model("Sub-Problem");
    
    z = sub.addVars(G.edges, vtype=GRB.BINARY);
    x = sub.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    
    sub.addConstrs(gp.quicksum(x[v,u] for u in G.successors(v)) 
                 - gp.quicksum(x[u,v] for u in G.predecessors(v)) == 0 for v in G.nodes);
       
    for i,j in G.edges:
        sub.addConstr(x[i,j] <= G.edges[i,j]['capacity']*(1-gammaStar[i,j]));
        if G.edges[i,j]['special'] == 0:
            sub.addConstr(z[i,j] == 0);
        else:
            sub.addConstr(x[i,j] >= (1/M)*z[i,j]);
                
    sub.addConstr(x[t,s] - xBar_ts >= 0);
          
            
        
    sub.setObjective(gp.quicksum(z[i,j]*G.edges[i,j]['special'] for i,j in G.edges), GRB.MAXIMIZE);
    sub.update();
    sub.setParam("OutputFlag", 0);
    sub.optimize();
    
    objVal = sub.objVal;
    UB = objVal;
    #print('UB (Sub): %g' %UB);
    
    
    for i,j in G.edges:
        xStar[i,j] = x[i,j].x;
        zStar[i,j] = z[i,j].x;
        #if i == t:
            #print('Flow (Sub): %g' % xStar[i,j]);    
    
    
    return xStar, zStar, UB;

In [4]:
# Naive Approach to check optimality

def naive_approach(G, s, t, UB):
    
    UB_min = UB;
    
    naive = gp.Model("OptimalityCheck");

    g = naive.addVars(G.edges, vtype=GRB.BINARY);
    x = naive.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);

    lst = list(itertools.product([0, 1], repeat=len(G.edges)));

    ctr_l = 0;
    for l in lst:
        ctr_e = 0;
        sum_g = 0;
        for i,j in G.edges:
            g[i,j] = l[ctr_e];
            ctr_e = ctr_e + 1;
            sum_g = sum_g + g[i,j]*G.edges[i,j]['cost'];
        
        if sum_g <= budget[b]:
            naive.remove(naive.getConstrs());
            naive.update();  
            naive.addConstrs(gp.quicksum(x[j,i] for i in G.successors(j)) 
                             - gp.quicksum(x[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
            for i,j in G.edges:
                naive.addConstr(x[i,j] - (1-g[i,j])*G.edges[i,j]['capacity'] <= 0);
            
            naive.setObjective(x[t,s], GRB.MAXIMIZE);
            naive.setParam("OutputFlag", 0);
            naive.optimize();
            
            obj = naive.objVal;
            
            x_prime, z_prime, UB_prime = Sub(g, x[t,s].x, G, t, s, UB);
            #print('UB (Naive - Sub):', UB_prime);
            
            if UB_min < UB_prime:
                UB_min = UB_prime;
            
    if (UB_min < UB):
        print('Not optimal');
    else:
        print('Optimal');
        
    

In [5]:
# Main

#'''
# Creating Results Files ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

file_summary = open("Results_Summary.csv", "w");

file_summary.write('Instance\t, Budget\t, Rate\t, Nodes\t, Traffick\t, Bottoms\t, Victims\t,');
file_summary.write('Obj_value\t, Flow\t, Int_Traf\t, Int_Bot\t, Int_Vic\t,  Run_Time\n')
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#'''

num = 1;
budget = [0,4,8];
rate = [1];
network = {};
z = {};
xBar_ts = {};



# Input -- Reading the networks
for n in range(num):
    #network[n] = 'Network'+str(n+1);
    network[n] = 'Network2_Test';
    
    for b in range(len(budget)):       
        
        for r in range(len(rate)):
            
            print('\nNetwork', n+1, ', budget', budget[b], ', rate', rate[r]);
            
            start_time = time.time(); 
            
            with open(network[n]+'.csv', newline='') as f:
                reader = csv.reader(f);
                row1 = next(reader);
                s = int(row1[0]);
                t = int(row1[1]);
    
                G = nx.DiGraph();
                data = pd.read_csv(network[n]+'.csv',skiprows=1, header=None);
                n_edge = len(data.index+1);
        
                for i in range(n_edge): 
                    G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], trafficker=data.iat[i,5], 
                    bottom=data.iat[i,6], victim=data.iat[i,7]);
                    
                for i,j in G.edges:
                    if G.edges[i,j]['special'] == 1:
                        z[i,j] = 1;
                    else:
                        z[i,j] = 0;
                        
                    if i == s:
                        G.edges[i,j]['capacity'] = math.floor(rate[r]*G.edges[i,j]['capacity']);
                    
                A = 0;
                T = 0;
                B = 0;
                V = 0;
                U = 0;

                for i,j in G.edges:
                    if G.edges[i,j]['special'] == 1:
                        A = A + 1;
                    if G.edges[i,j]['trafficker'] == 1:
                        T = T + 1;
                    elif G.edges[i,j]['bottom'] == 1:
                        B = B + 1;
                    elif G.edges[i,j]['victim'] == 1:
                        V = V + 1;
                        U = U + G.edges[i,j]['capacity'];


            # Tau for the linearization of x_{ts}*delta
            tau = range(math.ceil(math.log(U))+1); #[];
                
                
        # C&CG       
            LB = 0;
            UB = 1e6;
            eps = 0.0001;
            
            Z = [];
            Z.append(z);        
     
            Gamma = [];
            
            gammaStar = {};
            xBarStar = {};
            
            it = 1;           
            while (UB - LB > eps):           
                gammaStar, xBar_ts, LB = Master(Z, budget[b], G, tau, s, t, it);
                print('LB (Master):', "{:.2f}".format(LB));
                print('xBar[t,s] (Master):', xBar_ts);
                xStar, zStar, UB = Sub(gammaStar, xBar_ts, G, t, s, UB);
                print('UB (Sub):', UB);
                Z.append(zStar);
                it = it + 1;
            
            
            obj = UB;
            sol = A - UB;
            
            # Only run the Naive function if number of edges is small!
            if (G.number_of_edges() <= 20):
                naive_approach(G, s, t, UB);


#'''  
# Printing results to file   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            
            file = open('results_'+network[n]+'_b'+str(budget[b])+'_r'+str(rate[r])+'.txt', "w");
            
            file.write('Number of Nodes: %g' % (T+B+V) +'\n');
            file.write('Number of Traffickers: %g' % T +'\n');
            file.write('Number of Bottoms: %g' % B +'\n');
            file.write('Number of victims: %g' % (V) +'\n');
            file.write('Budget: %g' % budget[b] +'\n\n');
            
            file.write('Number of interdicted edges in cal_A: %g' % sol +'\n');
            file.write('Number of uninterdicted edges in cal_A: %g' % obj +'\n');
            file.write('Max-Flow: %g' % xStar[t, s] +'\n');
            file.write('Run-time: %s' % "{:.2f}".format((time.time() - start_time)) + ' sec');
            file.write('\n\n');    
            file.write('Trafficker Capacity: \n');
            
            for i,j in G.edges:
                if i==s:
                    file.write(str(i)+', '+str(j)+': '+ str(G.edges[i,j]['capacity']) +'\n');
            
            int_T = 0;
            int_B = 0;
            int_V = 0;
            file.write('\nInterdiction plan: \n');
                
            for i, j in G.edges: 
                if gammaStar[i,j] > 0.1: 
                    if G.edges[i,j]['trafficker'] == 1:
                        file.write('Trafficker: %s' % j +'\n');
                        int_T = int_T + 1;
                    elif G.edges[i,j]['bottom'] == 1:
                        file.write('Bottom: %s' % j +'\n');
                        int_B = int_B + 1;
                    elif G.edges[i,j]['victim'] == 1:
                        file.write('Victim: %s' % i +'\n');
                        int_V = int_V + 1;
                        
            
            file.write('\n');
            file.write('Victim nodes with flow' +'\n');    
                
            for i, j in G.edges:
                if G.edges[i,j]['special'] == 1:
                    if xStar[i,j] > 0.0000001:
                        file.write('victim:'+ str(i)+'\t Flow: %f' % xStar[i,j] +'\n');
                
            file_summary.write(network[n]+'_b'+str(budget[b])+'_r'+str(rate[r])+','+str(budget[b])+','
                               +str(rate[r])+',' +str(T+B+V)+','+str(T)+','+str(B)+','+str(V)+','
                               +str(obj)+','+str("{:.2f}".format(xStar[t, s]))+','+str(int_T)+','
                               +str(int_B)+','+str(int_V)+','
                               +str("{:.2f}".format((time.time() - start_time)) + ' sec')+'\n');
            
            
            file.write('\n');
            file.close();

                
file_summary.close();              
#'''    


Network 1 , budget 0 , rate 1
Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-30
Iteration: 1
LB (Master): 6.00
xBar[t,s] (Master): 0.0
UB (Sub): 6.0
Optimal

Network 1 , budget 4 , rate 1
Iteration: 1
LB (Master): 4.00
xBar[t,s] (Master): 0.0
UB (Sub): 4.0
Optimal

Network 1 , budget 8 , rate 1
Iteration: 1
LB (Master): 2.00
xBar[t,s] (Master): 0.0
UB (Sub): 2.0
Optimal
